NO DISTORTION PARAMETERS YET

In [15]:
import rosbag
from geometry_msgs.msg import Point
import pandas as pd
import os
import cv2
from cv_bridge import CvBridge

In [62]:
dataset = 'translation_1_z'

In [63]:
try:
    os.mkdir('/home/MSc/data/Texts/{}'.format(dataset))
except:
    print 'Dir already exists'

Dir already exists


In [64]:
# The bag file should be in the same directory as your terminal
bag = rosbag.Bag('/home/MSc/data/Rosbags/{}.bag'.format(dataset))

In [65]:
topic = '/cam0/image_raw'
column_names = ['timestamp','path']
df = pd.DataFrame(columns=column_names)

i = 0
bridge = CvBridge()

for topic, msg, t in bag.read_messages(topics=topic):
    df = df.append(
        {'timestamp': msg.header.stamp,
        'path':  "images/frame{0:04d}.jpg".format(i)},
        ignore_index=True
    )
    cv_img = bridge.imgmsg_to_cv2(msg, desired_encoding="passthrough")
    cv2.imwrite(os.path.join('/home/MSc/data/Texts/', "{0}/images/frame{1:04d}.png".format(dataset, i)), cv_img)

    i += 1
    
df.to_csv('/home/MSc/data/Texts/{}/images.txt'.format(dataset), index = False, header = False)    

In [5]:
topic = '/imu'
column_names = ['timestamp','ax', 'ay', 'az','gx','gy','gz']
df = pd.DataFrame(columns=column_names)
#df.set_index('timestamp')

for topic, msg, t in bag.read_messages(topics=topic):
    
    time = msg.header.stamp
    
    g_x = msg.angular_velocity.x
    g_y = msg.angular_velocity.y
    g_z = msg.angular_velocity.z
    
    a_x = msg.linear_acceleration.x
    a_y = msg.linear_acceleration.y
    a_z = msg.linear_acceleration.z

    df = df.append(
        {'timestamp': time,
        'ax': a_x,
        'ay': a_y,
        'az': a_z,
        'gx': g_x,
        'gy': g_y,
        'gz': g_z},
        ignore_index=True
    )

df.to_csv('/home/MSc/data/Texts/{}/imu.txt'.format(dataset), index = False, header = False)

In [6]:
topic = '/cam0/camera_info'
column_names = ['fx','fy', 'cx', 'cy','k1','k2','p1', 'p2', 'k3']
df = pd.DataFrame(columns=column_names)

for topic, msg, t in bag.read_messages(topics=topic):
    continue

df = df.append(
    {'fx': msg.K[0],
    'fy':  msg.K[4],
    'cx':  msg.K[2],
    'cy':  msg.K[5],
    'k1': 0,
    'k2': 0,
    'p1': 0,
    'p2': 0,
    'k3': 0},
    ignore_index=True
)

df.to_csv('/home/MSc/data/Texts/{}/calib.txt'.format(dataset), index = False, header = False)

In [7]:
topic = '/cam0/pose'
column_names = ['timestamp','x', 'y', 'z','qx','qy','qz', 'qw']
df = pd.DataFrame(columns=column_names)

for topic, msg, t in bag.read_messages(topics=topic):
    df = df.append(
        {'timestamp': msg.header.stamp,
        'x':  msg.pose.position.x,
        'y':  msg.pose.position.y,
        'z':  msg.pose.position.z,
        'qx': msg.pose.orientation.x,
        'qy': msg.pose.orientation.y,
        'qz': msg.pose.orientation.z,
        'qw': msg.pose.orientation.w},
        ignore_index=True
    )



df.to_csv('/home/MSc/data/Texts/{}/groundtruth.txt'.format(dataset), index = False, header = False)